In [2]:
import numpy as np
from time import time

### 1 Why not use loop
---

#### 1.1 Arrays are mutable

In [3]:
x_array = np.arange(5)
x_array

array([0, 1, 2, 3, 4])

In [4]:
x_array[0] = 99
x_array

array([99,  1,  2,  3,  4])

In [5]:
x_array[3:6] = [99,99] # this is permissable since the array returned by the slice is of the same shape as the assignment array
x_array

array([99,  1,  2, 99, 99])

#### 1.2 Loops are slow

In [6]:
bigmatrix = np.random.random((1000,2000))
bigmatrix.size

2000000

In [7]:
# A loop based computation of the sum of the max of all rows

def sum_of_max_loop(bigmatrix):
    
    m,n = bigmatrix.shape
    
    maxValues = np.zeros(shape=(m,))
    
    for i in range(m):
        for j in range(n):
            if(bigmatrix[i,j] > maxValues[i]):
                maxValues[i] = bigmatrix[i,j]
    
    sum_of_row_maxes = 0
    for i in range(m):
        sum_of_row_maxes += maxValues[i]
    
    return sum_of_row_maxes

In [8]:
start = time()
sum_of_row_maxes_loop = sum_of_max_loop(bigmatrix=bigmatrix)
duration_loop = time() - start

print(f"sum_of_max_loop = {sum_of_row_maxes_loop}")
print(f"took: {duration_loop:.4f} seconds.")

sum_of_max_loop = 999.4986820257034
took: 0.4572 seconds.


#### 1.3 Vectorized computation

In [9]:
def sum_of_max_vec(bigmatrix):
    maxValues = np.max(bigmatrix, axis=1)
    return sum(maxValues)

In [10]:
start = time()
sum_of_row_maxes_vec = sum_of_max_vec(bigmatrix)
duration_vec = time()-start

print(f"sum_of_max_vec = {sum_of_row_maxes_vec}")
print(f"took: {duration_vec:.4f} seconds")

sum_of_max_vec = 999.4986820257034
took: 0.0017 seconds


In [11]:
print(f"Therefore vectorized computation is roughly {duration_loop/duration_vec} times faster")

Therefore vectorized computation is roughly 275.81962025316454 times faster


In [12]:
import matplotlib.pyplot as plt

x = np.arange(start=0, stop=100, step=1)
loop_durations = np.zeros_like(x, dtype=float)
vec_durations = np.zeros_like(x, dtype=float)


for i in range(100):
    
    start = time()
    sum_of_max_loop(bigmatrix)
    duration_loop = time() - start
    loop_durations[i] = duration_loop #round(duration_loop, ndigits=4)
    
    start = time()
    sum_of_max_vec(bigmatrix)
    duration_vec = time() - start
    vec_durations[i] = duration_vec #round(duration_vec, ndigits=4)

plt.figure(figsize=(8, 5))
plt.plot(x, loop_durations, label="Loop Durations", color='blue')
plt.plot(x, vec_durations, label="Vectorized Durations", color='red')

# Set the y-axis to a logarithmic scale
plt.yscale('log')

# Add labels and title
plt.xlabel('Iteration')
plt.ylabel('Duration (seconds)')
plt.title('Comparison of Loop and Vectorized Durations (Log Scale)')

# Add a legend
plt.legend()

# Display the plot
plt.show()

KeyboardInterrupt: 

- values closer to 0 appear appear on the plot as increasingly negative powers of 10
    - log_10(0.1) = -1 so the plot would label this value 10^-1
    - log_10(0.01) = -2 so the plot would label this value 10^-2
    - etc...

### 2 UFunc: Universal Functions
---

#### 2.1 Unary ufunc

- ufuncs are applied to the *entire* array efficiently utilizing highly optimized low level execution plans
- a unary function is a function with a single input
- when applied to a multidimensional numpy array, all elements are mapped to an output array organized in the same shape as the input array

In [13]:
# arbitrarily shaped ndarray

x = np.random.uniform(-100, 100, (3,2,2))
x = x.round(2)
print(f"{x}\n\nshape = {x.shape}")

[[[ 66.68 -59.72]
  [ 55.01 -22.08]]

 [[-11.    52.17]
  [ 78.92  80.02]]

 [[-71.97  19.14]
  [ 65.64  71.77]]]

shape = (3, 2, 2)


In [24]:
print('x =\n', x)
print(f'|x| = \n{np.abs(x)}\n')     # absolute value of each element
print('x + 1 =\n', x + 1)      # addition
print('x - 1 =\n', x - 1)      # subtraction
print('x * 2 =\n', x * 2)      # multiplication
print('x // 2 =\n', x // 2)     # integer division
print('x ** 2 =\n', x ** 2)     # square
print('x % 2 =\n', x % 2)      # modulo  
print('1 / x =\n', 1 / x)      # division
print(f"x / (100 * 2 * pi) = \n{x / 100 * 2 * np.pi}\n")    # chain of operations
print(f"sin(x) = \n{np.sin(x)}\n")      # sine of x
print(f"x**2 = \n{np.power(x,2)}\n")    # x**2
print(f"sort(x) = \n{np.sort(x)}\n")    # sort x
print(f"sign(x) = \n{np.sign(x)}\n")    # replace each element with 1 or -1 based on its sign

x =
 [[[ 66.68 -59.72]
  [ 55.01 -22.08]]

 [[-11.    52.17]
  [ 78.92  80.02]]

 [[-71.97  19.14]
  [ 65.64  71.77]]]
|x| = 
[[[66.68 59.72]
  [55.01 22.08]]

 [[11.   52.17]
  [78.92 80.02]]

 [[71.97 19.14]
  [65.64 71.77]]]

x + 1 =
 [[[ 67.68 -58.72]
  [ 56.01 -21.08]]

 [[-10.    53.17]
  [ 79.92  81.02]]

 [[-70.97  20.14]
  [ 66.64  72.77]]]
x - 1 =
 [[[ 65.68 -60.72]
  [ 54.01 -23.08]]

 [[-12.    51.17]
  [ 77.92  79.02]]

 [[-72.97  18.14]
  [ 64.64  70.77]]]
x * 2 =
 [[[ 133.36 -119.44]
  [ 110.02  -44.16]]

 [[ -22.    104.34]
  [ 157.84  160.04]]

 [[-143.94   38.28]
  [ 131.28  143.54]]]
x // 2 =
 [[[ 33. -30.]
  [ 27. -12.]]

 [[ -6.  26.]
  [ 39.  40.]]

 [[-36.   9.]
  [ 32.  35.]]]
x ** 2 =
 [[[4446.2224 3566.4784]
  [3026.1001  487.5264]]

 [[ 121.     2721.7089]
  [6228.3664 6403.2004]]

 [[5179.6809  366.3396]
  [4308.6096 5150.9329]]]
x % 2 =
 [[[0.68 0.28]
  [1.01 1.92]]

 [[1.   0.17]
  [0.92 0.02]]

 [[0.03 1.14]
  [1.64 1.77]]]
1 / x =
 [[[ 0.014997   -0.0167

### 2.2 Binary ufunc
---

- binary ufunc processes two arrays A and B in their entirety
- input arrays must have the same or boradcast-compatible shapes
- the output array has the same (post-broadcast if applicable) shape as the input arrays
- each element in the output array is calculated using pairs of corresponding elements in the input arrays

In [17]:
A = np.arange(1, 1+12).reshape(3,4)
B = np.arange(10, 10+12).reshape(3,4) # this is a good way to ensure each array will have +# elements

print("A =\n", A)
print("B =\n", B)

A =
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
B =
 [[10 11 12 13]
 [14 15 16 17]
 [18 19 20 21]]


In [18]:

print("A + B = \n", A+B)
print("B / A = \n", B/A)
print("B / A (using np.divide) = \n", np.divide(B,A))
print("B // A =\n", B // A)
print("B // A (using np.floor_divide) = \n", np.floor_divide(B,A))
print("B ** A =\n", B ** A)
print("B ** A (using np.power) =\n", np.power(B,A))

x = np.arange(-2,3)
y = np.random.randn(5)
print('x =', x)
print('y =', y, '\n')
print('np.add(x,y) =', np.add(x,y))                # element-wise addition       x + y
print('np.subtract(x,y) =', np.subtract(x,y))      # element-wise subtraction    x - y
print('np.multiply(x,y) =', np.multiply(x,y))      # element-wise multiplication x * y
print('np.divide(x,y) =', np.divide(x,y))          # element-wise division       x / y
print('np.maximum(x,y) =', np.maximum(x,y))        # element-wise maximum        max(x,y)

A + B = 
 [[11 13 15 17]
 [19 21 23 25]
 [27 29 31 33]]
B / A = 
 [[10.          5.5         4.          3.25      ]
 [ 2.8         2.5         2.28571429  2.125     ]
 [ 2.          1.9         1.81818182  1.75      ]]
B / A (using np.divide) = 
 [[10.          5.5         4.          3.25      ]
 [ 2.8         2.5         2.28571429  2.125     ]
 [ 2.          1.9         1.81818182  1.75      ]]
B // A =
 [[10  5  4  3]
 [ 2  2  2  2]
 [ 2  1  1  1]]
B // A (using np.floor_divide) = 
 [[10  5  4  3]
 [ 2  2  2  2]
 [ 2  1  1  1]]
B ** A =
 [[              10              121             1728            28561]
 [          537824         11390625        268435456       6975757441]
 [    198359290368    6131066257801  204800000000000 7355827511386641]]
B ** A (using np.power) =
 [[              10              121             1728            28561]
 [          537824         11390625        268435456       6975757441]
 [    198359290368    6131066257801  204800000000000 735582751138664

#### Important Statistical Methods

In [ ]:
y = np.array([-3.2, -1.4, 0.4, 2.5, 3.4])    
print('y =', y, '\n')

print("Min =", np.min(y))             # min 
print("Max =", np.max(y))             # max 
print("Average =", np.mean(y))        # mean/average
print("Std deviation =", np.std(y))   # standard deviation
print("Sum =", np.sum(y))             # sum 

#### Important Linear Algebra Operations

In [ ]:
X = np.random.randn(2,3)                         # create a 2 x 3 random matrix
print('X =\n', X, '\n')
print('Transpose of X, X.T =\n', X.T, '\n')      # matrix transpose operation X^T

y = np.random.randn(3) # random vector 
print('y =', y, '\n')

print('Matrix-vector multiplication')
print('X.dot(y) =\n', X.dot(y), '\n')            # matrix-vector multiplication  X * y

print('Matrix-matrix product')
print('X.dot(X.T) =', X.dot(X.T))        # matrix-matrix multiplication  X * X^T
print('\nX.T.dot(X) =\n', X.T.dot(X))      # matrix-matrix multiplication  X^T * X

In [ ]:
X = np.random.randn(5,3)
print('X =\n', X, '\n')

C = X.T.dot(X)               # C = X^T * X is a square matrix
print('C = X.T.dot(X) =\n', C, '\n')

invC = np.linalg.inv(C)      # inverse of a square matrix
print('Inverse of C = np.linalg.inv(C)\n', invC, '\n')

detC = np.linalg.det(C)      # determinant of a square matrix
print('Determinant of C = np.linalg.det(C) =', detC)

S, U = np.linalg.eig(C)      # eigenvalue S and eigenvector U of a square matrix
print('Eigenvalues of C =\n', S)
print('Eigenvectors of C =\n', U)

### 2.3 Broadcasting
---

- if $A.shape \neq B.shape$
    1. runtime error

    2. **manually** adjust the shape using `np.reshape` or `np.repeat`
    
    3. **broadcasting** is applied using the following rules:
    
        - any axis of size 1 will automatically be repeated so that shapes match
    
        - if `len(A.shape)` $\neq$ `len(B.shape)` then np.newaxis will be used to add a new axis that will then be repeated using np.repeat

            - np.newaxis will only ***append*** new axes so all trailing dimensions must match 

In [19]:
A

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [20]:
B = np.array([10,20,30,40])

In [21]:
A + B

array([[11, 22, 33, 44],
       [15, 26, 37, 48],
       [19, 30, 41, 52]])

In [22]:
# first np.newaxis was automatically invoked to increase the dimension of B by 1
B[np.newaxis,:]
# since B.shape[axis = 1] == A.shape[axis = 1], np.repeat is used to set B.shape[axis=0] = A.shape[axis=0]
np.repeat(B[np.newaxis,:],3,axis=0)
# B has now been broadcasted appropriately allowing for the addition A + B to be possible
# A + np.repeat(B[np.newaxis,:],3,axis=0)

array([[10, 20, 30, 40],
       [10, 20, 30, 40],
       [10, 20, 30, 40]])